Anova_glove_250features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Glove/glove_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,-0.452040,0.801449,-0.798262,0.443883,0.107085,0.103052,-0.070691,-0.379046,-0.591434,...,-0.039812,-0.005104,0.110983,-0.309915,-0.583130,0.915560,0.093664,-0.893171,-0.866044,-1.140370
1,1,2.274216,0.203562,1.082171,0.972508,1.107539,-2.720579,-1.098134,1.448865,0.871496,...,-0.888288,-0.674572,-1.516089,-3.001942,-3.720266,-1.661635,-1.087040,2.048957,0.283239,1.962313
2,2,1.476509,-2.471183,1.064253,-1.066375,-1.525328,-3.141398,-1.153571,1.461766,-1.028248,...,-0.970296,-1.201938,-2.182499,-1.342782,-0.685782,-0.607378,0.306042,-0.600517,1.311579,0.343899
3,3,-0.175850,-0.780436,-0.191251,-0.077540,-1.142978,-0.608084,-0.284914,-0.296537,-1.177505,...,-0.892874,-0.545916,-0.500476,0.559545,0.477564,-0.516697,0.593033,1.137595,0.409999,0.869534
4,4,1.040033,0.283966,-1.215617,-0.242167,0.285401,0.326397,-0.517945,-0.848897,0.337202,...,0.027072,-0.235785,-0.262704,0.856284,0.246013,-0.099549,-0.088800,0.488470,-0.727014,1.020260


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.452040,0.801449,-0.798262,0.443883,0.107085,0.103052,-0.070691,-0.379046,-0.591434,0.245580,...,-0.039812,-0.005104,0.110983,-0.309915,-0.583130,0.915560,0.093664,-0.893171,-0.866044,-1.140370
1,2.274216,0.203562,1.082171,0.972508,1.107539,-2.720579,-1.098134,1.448865,0.871496,-0.228744,...,-0.888288,-0.674572,-1.516089,-3.001942,-3.720266,-1.661635,-1.087040,2.048957,0.283239,1.962313
2,1.476509,-2.471183,1.064253,-1.066375,-1.525328,-3.141398,-1.153571,1.461766,-1.028248,-0.847656,...,-0.970296,-1.201938,-2.182499,-1.342782,-0.685782,-0.607378,0.306042,-0.600517,1.311579,0.343899
3,-0.175850,-0.780436,-0.191251,-0.077540,-1.142978,-0.608084,-0.284914,-0.296537,-1.177505,-0.111262,...,-0.892874,-0.545916,-0.500476,0.559545,0.477564,-0.516697,0.593033,1.137595,0.409999,0.869534
4,1.040033,0.283966,-1.215617,-0.242167,0.285401,0.326397,-0.517945,-0.848897,0.337202,-1.011081,...,0.027072,-0.235785,-0.262704,0.856284,0.246013,-0.099549,-0.088800,0.488470,-0.727014,1.020260


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [15]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [16]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3407844634455358


**Decision Tree**

In [19]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [20]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [21]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [22]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [23]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [24]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-23-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.50082,0.50082,0.505948,0.50053,0.349237,0.349237,0.350364,0.348004


**Random Forest**

In [25]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [26]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [27]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [28]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9777    0.9695    0.9736       361
           2     0.9852    0.9656    0.9753       553
           3     0.9437    0.9869    0.9649       306

    accuracy                         0.9721      1220
   macro avg     0.9689    0.9740    0.9713      1220
weighted avg     0.9726    0.9721    0.9722      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3425    0.1592    0.2174       157
           2     0.4504    0.7848    0.5723       237
           3     0.2368    0.0692    0.1071       130

    accuracy                         0.4198       524
   macro avg     0.3432    0.3378    0.2989       524
weighted avg     0.3651    0.4198    0.3506       524



In [29]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-23-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.500820,0.500820,0.505948,0.500530,0.349237,0.349237,0.350364,0.348004
1,RandomForest,0.972131,0.972131,0.972589,0.972189,0.419847,0.419847,0.365062,0.350565


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5977    0.4238    0.4959       361
           2     0.5651    0.7613    0.6487       553
           3     0.6256    0.4477    0.5219       306

    accuracy                         0.5828      1220
   macro avg     0.5961    0.5443    0.5555      1220
weighted avg     0.5899    0.5828    0.5717      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3119    0.2166    0.2556       157
           2     0.4146    0.5738    0.4814       237
           3     0.2184    0.1462    0.1751       130

    accuracy                         0.3607       524
   macro avg     0.3150    0.3122    0.3041       524
weighted avg     0.3352    0.3607    0.3378       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518033,0.518033,0.612516,0.437990,0.438931,0.438931,0.389072,0.340146
1,RandomForest,0.972951,0.972951,0.973012,0.972945,0.431298,0.431298,0.380718,0.369685
2,Adaboost,0.582787,0.582787,0.589901,0.571693,0.360687,0.360687,0.335175,0.337778


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9734    0.9114    0.9413       361
           2     0.8990    0.9819    0.9386       553
           3     0.9856    0.8954    0.9384       306

    accuracy                         0.9393      1220
   macro avg     0.9527    0.9296    0.9394      1220
weighted avg     0.9427    0.9393    0.9394      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3333    0.2675    0.2968       157
           2     0.4496    0.6582    0.5342       237
           3     0.2157    0.0846    0.1215       130

    accuracy                         0.3989       524
   macro avg     0.3329    0.3368    0.3175       524
weighted avg     0.3567    0.3989    0.3607       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518033,0.518033,0.612516,0.437990,0.438931,0.438931,0.389072,0.340146
1,RandomForest,0.972951,0.972951,0.973012,0.972945,0.431298,0.431298,0.380718,0.369685
2,Adaboost,0.582787,0.582787,0.589901,0.571693,0.360687,0.360687,0.335175,0.337778
3,GBM,0.939344,0.939344,0.942734,0.939367,0.398855,0.398855,0.356718,0.360722


**SVM Model**

In [30]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [31]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [32]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [33]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5541    0.6953    0.6167       361
           2     0.7506    0.5497    0.6347       553
           3     0.5856    0.6928    0.6347       306

    accuracy                         0.6287      1220
   macro avg     0.6301    0.6459    0.6287      1220
weighted avg     0.6511    0.6287    0.6294      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2906    0.3758    0.3278       157
           2     0.5030    0.3502    0.4129       237
           3     0.2500    0.3000    0.2727       130

    accuracy                         0.3454       524
   macro avg     0.3479    0.3420    0.3378       524
weighted avg     0.3766    0.3454    0.3526       524



In [34]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-23-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.500820,0.500820,0.505948,0.500530,0.349237,0.349237,0.350364,0.348004
1,RandomForest,0.972131,0.972131,0.972589,0.972189,0.419847,0.419847,0.365062,0.350565
2,SVM,0.628689,0.628689,0.651082,0.629364,0.345420,0.345420,0.376620,0.352636


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4913    0.5457    0.5171       361
           2     0.5895    0.6908    0.6361       553
           3     0.5439    0.3039    0.3899       306

    accuracy                         0.5508      1220
   macro avg     0.5415    0.5135    0.5144      1220
weighted avg     0.5490    0.5508    0.5392      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2924    0.3185    0.3049       157
           2     0.4487    0.4979    0.4720       237
           3     0.2556    0.1769    0.2091       130

    accuracy                         0.3645       524
   macro avg     0.3322    0.3311    0.3287       524
weighted avg     0.3539    0.3645    0.3567       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518033,0.518033,0.612516,0.437990,0.438931,0.438931,0.389072,0.340146
1,RandomForest,0.972951,0.972951,0.973012,0.972945,0.431298,0.431298,0.380718,0.369685
2,Adaboost,0.582787,0.582787,0.589901,0.571693,0.360687,0.360687,0.335175,0.337778
3,GBM,0.939344,0.939344,0.942734,0.939367,0.398855,0.398855,0.356718,0.360722
4,SVM,0.637705,0.637705,0.638753,0.631954,0.358779,0.358779,0.346085,0.350952
5,KNN,0.550820,0.550820,0.548989,0.539150,0.364504,0.364504,0.353938,0.356702


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3274    0.7175    0.4497       361
           2     0.5641    0.1591    0.2482       553
           3     0.3223    0.2876    0.3040       306

    accuracy                         0.3566      1220
   macro avg     0.4046    0.3881    0.3340      1220
weighted avg     0.4334    0.3566    0.3218      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3123    0.6624    0.4245       157
           2     0.5181    0.1814    0.2687       237
           3     0.2778    0.2308    0.2521       130

    accuracy                         0.3378       524
   macro avg     0.3694    0.3582    0.3151       524
weighted avg     0.3968    0.3378    0.3113       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518033,0.518033,0.612516,0.437990,0.438931,0.438931,0.389072,0.340146
1,RandomForest,0.972951,0.972951,0.973012,0.972945,0.431298,0.431298,0.380718,0.369685
2,Adaboost,0.582787,0.582787,0.589901,0.571693,0.360687,0.360687,0.335175,0.337778
3,GBM,0.939344,0.939344,0.942734,0.939367,0.398855,0.398855,0.356718,0.360722
4,SVM,0.637705,0.637705,0.638753,0.631954,0.358779,0.358779,0.346085,0.350952
5,KNN,0.550820,0.550820,0.548989,0.539150,0.364504,0.364504,0.353938,0.356702
6,GNB,0.356557,0.356557,0.433434,0.321816,0.337786,0.337786,0.396808,0.311282
